In [1]:
import cv2
import numpy as np

def process_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    canny = cv2.Canny(blur, 50, 150)
    return canny

def region_of_interest(image):
    height = image.shape
    polygons = np.array([[(200, height), (1100, height), (550, 250)]])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    return line_image

image = cv2.imread('Curve3.jpg')
lane_image = np.copy(image)
canny = process_image(lane_image)
cropped_image = region_of_interest(canny)
lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)
line_image = display_lines(lane_image, lines)
combo_image = cv2.addWeighted(lane_image, 0.8, line_image, 1, 1)
cv2.imshow('result', combo_image)
cv2.waitKey(0)

C:\Users\Yatharth\AppData\Local\Temp\ipykernel_10480\180737560.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  polygons = np.array([[(200, height), (1100, height), (550, 250)]])


error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'fillPoly'
> Overload resolution failed:
>  - Can't parse 'pts'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pts'. Sequence item with index 0 has a wrong type


In [3]:
import cv2
import numpy as np

# Load the image
img = cv2.imread('Curve3.jpg')

# Resize the image to a smaller size
img = cv2.resize(img, (640, 480))

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Canny edge detection
edges = cv2.Canny(gray, 50, 150)

# Apply Gaussian blur to reduce noise
blur = cv2.GaussianBlur(edges, (5, 5), 0)

# Apply Hough transform to detect lines
lines = cv2.HoughLinesP(blur, 1, np.pi/180, 50, minLineLength=100, maxLineGap=10)

# Define empty arrays for left and right lanes
left_lane = []
right_lane = []

# Loop through detected lines and classify them as left or right lane
for line in lines:
    x1, y1, x2, y2 = line[0]
    slope = (y2 - y1) / (x2 - x1)
    if slope < 0:
        left_lane.append((x1, y1))
        left_lane.append((x2, y2))
    else:
        right_lane.append((x1, y1))
        right_lane.append((x2, y2))

# Fit a polynomial curve to the left and right lanes
if len(left_lane) > 0:
    left_fit = np.polyfit([x[0] for x in left_lane], [x[1] for x in left_lane], 1)
else:
    left_fit = None
if len(right_lane) > 0:
    right_fit = np.polyfit([x[0] for x in right_lane], [x[1] for x in right_lane], 1)
else:
    right_fit = None

# Create an empty image for visualizing the lanes
lane_image = np.zeros_like(img)

# Draw the left and right lanes on the image
if left_fit is not None:
    for y in range(img.shape[0]):
        x = int((y - left_fit[1]) / left_fit[0])
        if x >= 0 and x < img.shape[1]:
            lane_image[y, x, :] = [0, 255, 0]
if right_fit is not None:
    for y in range(img.shape[0]):
        x = int((y - right_fit[1]) / right_fit[0])
        if x >= 0 and x < img.shape[1]:
            lane_image[y, x, :] = [0, 0, 255]

# Overlay the lanes on the original image
result = cv2.addWeighted(img, 1, lane_image, 1, 0)

# Display the result
cv2.imshow('result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()
